In [14]:
# basic packages:

%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import sys

sys.path.insert( 0, '../tim')
from get_data import *

# scikit-learn modules
from sklearn.preprocessing import scale
from sklearn.model_selection import cross_val_score

## Data in the log basis

In [11]:
def log_basis( feature_vector ):
    return np.log10( feature_vector )

def log_transform_features(feature_matrix, features_to_transform):
    transformed_features  = []
    split_features = np.split( feature_matrix, feature_matrix.shape[1], axis=1 )
    for feature_num in list(range(len( split_features ))):
        feature = split_features[ feature_num ]
        if feature_num in features_to_transform:
            feature = log_basis( feature )
        transformed_features.append( feature )
    concat_features_array = np.concatenate( transformed_features, axis=1 )
    concat_features_array = scale(mean_fill_nan_entries( concat_features_array, is_df=False ))
    return concat_features_array

In [18]:
# Load Raw Data
raw_data_arrays = get_raw_data()

# Functions to Perform Log10 Operation on Specific Feature Vectors 
features_to_transform = [0, 1, 2, 16, 17, 18, 19]

# Transform Raw Data Features
std_industrials_log_data = log_transform_features(raw_data_arrays[2], features_to_transform)

/home/siddhesh/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: RuntimeWarning: invalid value encountered in log10
  from ipykernel import kernelapp as app


## XGBoost

In [4]:
import xgboost as xgb

In [16]:
data = np.split( std_industrials_log_data, [1], axis=1 )
y_data = data[0]
x_data = data[1]

In [20]:
params = {'silent':1, 'objective':'reg:linear'}
dtrain = xbg.DMatrix(data=x_data, label=y_data)

xbg.cv(params, dtrain, nfold=5, metrics={'error'}, seed=0)

,test-error-mean,test-error-std,train-error-mean,train-error-std
0,-0.319459,0.044191,-0.324352,0.011287
1,-0.321716,0.045634,-0.324912,0.011446
2,-0.321612,0.045591,-0.325274,0.011598
3,-0.322690,0.046075,-0.325596,0.011641
4,-0.323444,0.046593,-0.325682,0.011640
5,-0.323844,0.047094,-0.325668,0.011505
6,-0.323844,0.047094,-0.325717,0.011529
7,-0.324031,0.047033,-0.325939,0.011741
8,-0.323927,0.046980,-0.325914,0.011749
9,-0.323927,0.046980,-0.325967,0.011790


In [22]:
model = xbg.XGBRegressor()
scores = cross_val_score(model, x_data, y_data, scoring='neg_mean_squared_error', cv=5)
# print('MSEs:', scores*-1)
print("Average MSE: %0.4f (+/- %0.4f)" % (-scores.mean(), scores.std()))

Average MSE: 0.1991 (+/- 0.1095)
